In [2]:
import csv
import numpy as np
import pandas as pd
import MDAnalysis
from MDAnalysis.analysis.hydrogenbonds import WaterBridgeAnalysis


In [3]:
# Define the construct and replica file pattern
construct = "8HDO_HID"  # Replace with your construct name
replica_pattern = {
    "gro": f"{construct}-start.rep_{{}}.gro",
    "xtc": f"{construct}-traj_centered_skip.rep_{{}}.xtc"
}

# Define the ligand and list of residues to analyze
ligand_selection = "resname L01"
residues_to_analyze = [252, 253, 254]  # Replace with your residue IDs

# Define ligand donors and acceptors
HDO_donors = ['N0I', 'N0E']
HDO_acceptors = ['O0J', 'N0R', 'O0X', 'S06']



In [4]:
# Prepare data storage
all_results = []
occupancy_data = {resid: [] for resid in residues_to_analyze}  # Store occupancy counts for each residue


In [6]:
for x in range(1, 4):  # Loop through replicas 1, 2, and 3
    print(f"Processing Replica {x}...")
    
    # Define file paths dynamically
    gro_file = replica_pattern["gro"].format(x)
    xtc_file = replica_pattern["xtc"].format(x)
    
    # Load the trajectory for this replica
    traj = MDAnalysis.Universe(gro_file, xtc_file)
    
    # Slice the last 90 frames
    traj.trajectory[-90:]
    num_frames = len(traj.trajectory)  # Number of frames in the sliced trajectory
    
    # Prepare results for this replica
    replica_results = {"Time (ps)": None}
    
    for resid in residues_to_analyze:
        residue_selection = f"resid {resid}"
        print(f"Analyzing residue {resid} vs ligand for Replica {x}...")
        
        # Perform WaterBridgeAnalysis
        w = WaterBridgeAnalysis(
            traj, ligand_selection, residue_selection,
            distance=3.5, angle=150, donors=HDO_donors, acceptors=HDO_acceptors
        )
        w.run()
        
        # Extract data (time and bridge count per frame)
        data = w.count_by_time()
        data_array = np.array(data)
        
        # Store time in "Time (ps)" only for the first residue in each replica
        if replica_results["Time (ps)"] is None:
            replica_results["Time (ps)"] = data_array[:, 0]
        
        # Store bridge counts for this residue
        bridge_counts = data_array[:, 1]
        replica_results[f"Residue {resid}"] = bridge_counts
        
        # Count frames with bridges and store for occupancy calculation
        frames_with_bridges = np.sum(bridge_counts > 0)
        occupancy_data[resid].append(frames_with_bridges / num_frames * 100)  # Occupancy as a percentage
    
    # Convert to DataFrame and add Replica information
    replica_df = pd.DataFrame(replica_results)
    replica_df["Replica"] = f"Replica {x}"  # Add replica identifier
    all_results.append(replica_df)

# Combine all replicas
final_results = pd.concat(all_results, ignore_index=True)

# Save to CSV
output_file = f"{construct}_water_bridges_all_replicas.csv"
final_results.to_csv(output_file, index=False)

Processing Replica 1...
Analyzing residue 252 vs ligand for Replica 1...
Analyzing residue 253 vs ligand for Replica 1...
Analyzing residue 254 vs ligand for Replica 1...
Processing Replica 2...
Analyzing residue 252 vs ligand for Replica 2...
Analyzing residue 253 vs ligand for Replica 2...
Analyzing residue 254 vs ligand for Replica 2...
Processing Replica 3...
Analyzing residue 252 vs ligand for Replica 3...
Analyzing residue 253 vs ligand for Replica 3...
Analyzing residue 254 vs ligand for Replica 3...


In [7]:
# Calculate and print occupancy for each residue
print("\nResidue Occupancy (%) Across All Replicas:")
for resid, occupancies in occupancy_data.items():
    average_occupancy = np.mean(occupancies)
    print(f"Residue {resid}: {average_occupancy:.2f}%")



Residue Occupancy (%) Across All Replicas:
Residue 252: 34.98%
Residue 253: 0.00%
Residue 254: 0.00%
